In [ ]:
# MIT License
# Copyright (c) 2025 Hendrik Meyer
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


Python version: 3.11.11 (main, Dec 11 2024, 10:25:04) [Clang 14.0.6 ]


import sys
print("Python version:", sys.version)

In [2]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

import pkg_resources

print("pandas version:", pkg_resources.get_distribution("pandas").version)
print("numpy version:", pkg_resources.get_distribution("numpy").version)
print("statsmodels version:", pkg_resources.get_distribution("statsmodels").version)
print("tabulate version:", pkg_resources.get_distribution("tabulate").version)



pandas version: 2.1.4
numpy version: 1.26.4
statsmodels version: 0.13.5
tabulate version: 0.9.0


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the CSV file
df_validation = pd.read_csv(
    '/your/path/reduced_validation_cohort.csv',
    sep=";", decimal=",", index_col=0
)

# Convert all columns to numeric types (in case some were read as strings)
df_validation = df_validation.apply(pd.to_numeric, errors='coerce')

# Multiply selected columns by 1000 to improve readability
columns_to_multiply = [
    'doseInduSufenBolus', 'doseInduSufenPerfu',
    'doseInduEtomidateBolus', 'doseInduRemifPerfu'
]
df_validation[columns_to_multiply] = df_validation[columns_to_multiply] * 1000

df_validation

,inductDurat,surgDurat,anestDurat,age,weight,height,bmi,sex,rcri,urgency,...,aucMAPunder65Anes,twaMAPunder65Anes,meanAnes,stdAnes,entropyAnes,trendAnes,kurtosisAnes,skewnessAnes,baselineMAP,AKI_bin
2,27,107,177,42,62,167,22,0,1,0,...,50,0.28,90.59,11.42,0.93,0.23,14.48,-0.84,104,0
3,20,328,450,56,57,161,22,0,1,0,...,241,0.53,72.52,9.56,0.90,-0.32,61.31,4.88,89,0
7,9,62,115,29,47,163,18,0,1,0,...,0,0.00,91.22,7.82,0.91,0.43,3.56,0.69,92,0
9,35,69,181,53,74,184,22,1,1,0,...,412,2.26,68.99,10.88,0.92,0.01,2.24,0.83,91,0
11,20,108,162,82,91,180,28,1,0,0,...,0,0.00,102.42,14.30,0.95,-0.05,2.82,0.38,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15474,16,408,483,79,57,163,21,0,2,0,...,1157,2.39,71.55,13.38,0.92,-0.01,4.84,0.56,101,0
15475,29,272,324,67,53,161,20,0,2,0,...,95,0.29,82.11,13.48,0.92,-0.06,4.84,0.92,88,1
15476,20,96,147,71,59,167,21,1,1,0,...,0,0.00,90.78,12.22,0.92,-0.43,7.18,1.71,95,0
15477,16,72,155,53,162,180,50,1,2,0,...,261,1.67,72.07,11.09,0.92,0.22,2.92,0.29,85,0


In [ ]:
df_training = pd.read_csv(
    '/your/path/_reduced_training_cohort.csv',
    sep=";",
    decimal=",",
    index_col=0
)

# Convert all columns to numeric types (in case some were imported as strings)
df_training = df_training.apply(pd.to_numeric, errors='coerce')

# Multiply selected columns by 1000 to improve readability
columns_to_multiply = [
    'doseInduSufenBolus', 'doseInduSufenPerfu',
    'doseInduEtomidateBolus', 'doseInduRemifPerfu'
]
df_training[columns_to_multiply] = df_training[columns_to_multiply] * 1000

# Display the first rows for inspection
df_training

,inductDurat,surgDurat,anestDurat,age,weight,height,bmi,sex,rcri,urgency,...,aucMAPunder65Anes,twaMAPunder65Anes,meanAnes,stdAnes,entropyAnes,trendAnes,kurtosisAnes,skewnessAnes,baselineMAP,AKI_bin
1,89,223,388,71,75,155,31,0,1,1,...,959,2.47,70.33,15.89,0.91,0.03,28.78,3.85,85,1
4,13,77,120,63,96,176,31,1,2,0,...,138,1.14,79.64,14.81,0.96,0.03,3.62,0.43,124,0
5,23,37,83,76,90,182,27,1,1,0,...,78,0.93,76.99,12.82,0.95,-0.44,4.01,0.39,102,0
6,25,132,194,77,109,178,34,1,0,0,...,393,2.02,71.08,12.10,0.91,0.18,4.49,0.66,89,1
8,11,39,76,72,73,180,23,1,0,0,...,331,4.30,71.83,21.75,0.90,-0.26,4.83,1.47,97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,8,151,194,90,65,164,24,0,0,1,...,162,0.83,75.28,13.05,0.97,0.03,8.94,1.75,130,1
15482,25,97,180,89,76,174,25,1,3,0,...,0,0.00,88.35,6.62,0.88,-0.12,4.05,0.76,95,0
15483,10,129,226,52,46,164,17,0,0,1,...,875,3.85,66.09,11.40,0.90,0.17,4.95,-0.64,47,0
15484,19,131,219,77,58,170,20,1,2,0,...,137,0.62,73.47,6.32,0.90,-0.25,4.65,-1.17,72,0


In [5]:
# Remove the six variables excluded due to multicollinearity
# Columns to be removed
exclude_columns = [
    'surgDurat',
    'weight',
    'doseInduSteroInf',
    'twaMAPunder65Anes',
    'diab',
    'medSevoExp',
    'decrease'
]

# Drop the specified columns from both dataframes
df_training = df_training.drop(columns=exclude_columns)
df_validation = df_validation.drop(columns=exclude_columns)

In [ ]:
# Align the training cohort with the balanced training dataset
df_training_balanced = pd.read_csv(
    '/your/path/devds_balanced_prep.csv',
    index_col=0
)

df_training_balanced

,inductDurat,anestDurat,age,height,bmi,doseInduEtomidateBolus,doseInduPropoBolus,doseInduPropoPerfu,doseInduRemifPerfu,doseInduSufenBolus,...,clinic_cat_GCH,clinic_cat_HCH,clinic_cat_HNO,clinic_cat_NCH,clinic_cat_PWC,clinic_cat_TCH,clinic_cat_UCH,clinic_cat_URO,clinic_cat_Other,AKI_bin
1,2.356439,1.471024,0.692161,-1.754782,0.849566,-0.292412,0.610232,-0.703219,-0.498742,0.314511,...,0,0,0,1,0,0,0,0,0,1
6,0.269437,0.117461,0.924346,0.653424,1.314565,2.947915,-1.877275,-0.703219,-0.498742,-0.458122,...,0,0,0,0,0,0,0,0,1,1
12,0.569062,0.167152,0.651565,-0.146975,1.009385,3.124487,-1.877275,-0.703219,-0.498742,-0.822588,...,0,1,0,0,0,0,0,0,0,1
21,0.725702,1.721061,0.257694,0.750935,1.164286,-0.292412,0.461161,1.400572,-0.498742,0.109578,...,0,0,0,0,0,0,0,0,0,1
26,0.868706,0.524818,0.162287,-0.878848,3.355773,-0.292412,0.349987,-0.703219,-0.498742,-0.250979,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,0.675129,2.050349,0.732190,0.847903,-0.233228,-0.292412,-1.877275,-0.703219,-0.498742,-0.406260,...,0,0,0,0,0,0,0,0,1,0
3788,0.622951,0.475168,0.810607,1.230460,0.154157,3.858284,-1.877275,-0.703219,-0.498742,0.006813,...,1,0,0,0,0,0,0,0,0,0
9793,1.473102,1.383533,1.104409,0.257825,-0.438701,3.353171,0.004161,-0.703219,-0.498742,0.722008,...,0,0,0,1,0,0,0,0,0,0
3630,1.473102,1.962352,-0.612921,-0.878848,2.257007,-0.292412,0.440570,1.454601,-0.498742,0.006813,...,0,0,0,1,0,0,0,0,0,0


In [7]:
# Keep only those rows in df_training that also appear in df_training_balanced
df_training = df_training[df_training.index.isin(df_training_balanced.index)]

df_training

,inductDurat,anestDurat,age,height,bmi,sex,rcri,urgency,asa,nyha,...,minMAPcumu5MinAnes,aucMAPunder65Anes,meanAnes,stdAnes,entropyAnes,trendAnes,kurtosisAnes,skewnessAnes,baselineMAP,AKI_bin
1,89,388,71,155,31,0,1,1,2,0,...,48,959,70.33,15.89,0.91,0.03,28.78,3.85,85,1
6,25,194,77,178,34,1,0,0,2,0,...,49,393,71.08,12.10,0.91,0.18,4.49,0.66,89,1
12,30,199,70,170,32,0,2,0,2,0,...,49,412,80.36,31.20,0.94,0.08,7.09,2.22,174,1
21,33,441,61,179,33,1,1,0,2,0,...,43,611,74.93,13.19,0.93,-0.09,6.24,0.87,126,1
26,36,239,59,163,51,0,1,0,2,0,...,68,0,107.83,13.90,0.84,-0.05,8.24,0.09,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15470,19,146,54,180,30,1,1,0,4,0,...,56,178,79.60,14.03,0.93,0.09,2.02,-0.17,103,1
15478,12,130,70,170,24,1,0,1,4,0,...,58,129,72.34,11.94,0.94,-0.16,4.16,1.11,101,1
15479,55,284,38,171,25,0,1,1,4,0,...,58,96,89.56,15.63,0.92,0.18,2.59,-0.10,94,1
15480,8,194,90,164,24,0,0,1,2,0,...,55,162,75.28,13.05,0.97,0.03,8.94,1.75,130,1


In [8]:
list(df_training.columns)

['inductDurat',
 'anestDurat',
 'age',
 'height',
 'bmi',
 'sex',
 'rcri',
 'urgency',
 'asa',
 'nyha',
 'chf',
 'cad',
 'cvd',
 'pad',
 'diab_bin',
 'liverCirrh',
 'ekg',
 'ltmACE',
 'ltmSartan',
 'ltmBB',
 'ltmCCB',
 'ltmBig',
 'ltmIns',
 'ltmDiu',
 'ltmStat',
 'iuc',
 'stomTube',
 'doseInduEtomidateBolus',
 'doseInduPropoBolus',
 'doseInduPropoPerfu',
 'doseInduRemifPerfu',
 'doseInduSufenBolus',
 'doseInduSufenPerfu',
 'doseInduThiopBolus',
 'doseSurgGelafInf',
 'doseSurgSteroInf',
 'maxSevoExp',
 'eGFRPreSurg',
 'medInduKateBolus_bin',
 'medSurgAtroBolus_bin',
 'medSurgKateBolus_bin',
 'clinic_cat_ACH',
 'clinic_cat_GCH',
 'clinic_cat_HCH',
 'clinic_cat_HNO',
 'clinic_cat_NCH',
 'clinic_cat_PWC',
 'clinic_cat_TCH',
 'clinic_cat_UCH',
 'clinic_cat_URO',
 'clinic_cat_Other',
 'minMAPcumu1MinAnes',
 'minMAPcumu5MinAnes',
 'aucMAPunder65Anes',
 'meanAnes',
 'stdAnes',
 'entropyAnes',
 'trendAnes',
 'kurtosisAnes',
 'skewnessAnes',
 'baselineMAP',
 'AKI_bin']

In [9]:
df_validation

,inductDurat,anestDurat,age,height,bmi,sex,rcri,urgency,asa,nyha,...,minMAPcumu5MinAnes,aucMAPunder65Anes,meanAnes,stdAnes,entropyAnes,trendAnes,kurtosisAnes,skewnessAnes,baselineMAP,AKI_bin
2,27,177,42,167,22,0,1,0,3,0,...,76,50,90.59,11.42,0.93,0.23,14.48,-0.84,104,0
3,20,450,56,161,22,0,1,0,2,0,...,57,241,72.52,9.56,0.90,-0.32,61.31,4.88,89,0
7,9,115,29,163,18,0,1,0,2,0,...,80,0,91.22,7.82,0.91,0.43,3.56,0.69,92,0
9,35,181,53,184,22,1,1,0,2,0,...,57,412,68.99,10.88,0.92,0.01,2.24,0.83,91,0
11,20,162,82,180,28,1,0,0,1,0,...,79,0,102.42,14.30,0.95,-0.05,2.82,0.38,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15474,16,483,79,163,21,0,2,0,3,0,...,41,1157,71.55,13.38,0.92,-0.01,4.84,0.56,101,0
15475,29,324,67,161,20,0,2,0,3,1,...,58,95,82.11,13.48,0.92,-0.06,4.84,0.92,88,1
15476,20,147,71,167,21,1,1,0,4,0,...,77,0,90.78,12.22,0.92,-0.43,7.18,1.71,95,0
15477,16,155,53,180,50,1,2,0,3,0,...,55,261,72.07,11.09,0.92,0.22,2.92,0.29,85,0


In [10]:
# Group variables into categories
groups = {
    
    # Demography
    "Demography": ['age', 'height', 'bmi', 'sex'],
    
    # Preoperative risk assessment
    "Preoperative risk assessment": [
        'rcri', 'urgency', 'asa', 'eGFRPreSurg', 'iuc', 'stomTube', 'baselineMAP'
    ],
    
    # Surgical department
    "Surgical department": [
        'clinic_cat_ACH', 'clinic_cat_GCH', 'clinic_cat_HCH', 'clinic_cat_HNO', 'clinic_cat_NCH',
        'clinic_cat_PWC', 'clinic_cat_TCH', 'clinic_cat_UCH', 'clinic_cat_URO', 'clinic_cat_Other'
    ],
    
    # Durations within the anaesthesia process
    "Duration of the anaesthesia process": ['inductDurat', 'anestDurat'],
    
    # Pre-existing conditions
    "Pre-existing conditions": [
        'nyha', 'chf', 'cad', 'cvd', 'pad', 'diab_bin', 'liverCirrh', 'ekg'
    ],
    
    # Continuous medication
    "Continuous medication": [
        'ltmACE', 'ltmSartan', 'ltmBB', 'ltmCCB', 'ltmBig', 'ltmIns', 'ltmDiu', 'ltmStat'
    ],
    
    # Anaesthetic agents
    "Anaesthesia": [
        'medInduKateBolus_bin', 'medSurgAtroBolus_bin', 'medSurgKateBolus_bin', 'doseInduEtomidateBolus',
        'doseInduPropoBolus', 'doseInduPropoPerfu', 'doseInduRemifPerfu',
        'doseInduSufenBolus', 'doseInduSufenPerfu', 'doseInduThiopBolus', 'doseSurgGelafInf',
        'doseSurgSteroInf', 'maxSevoExp'
    ],
    
    # MAP feature variables
    "MAP Features": [
        'minMAPcumu1MinAnes', 'minMAPcumu5MinAnes', 'aucMAPunder65Anes',
        'meanAnes', 'stdAnes', 'entropyAnes', 'trendAnes', 'kurtosisAnes', 'skewnessAnes'
    ]
}


In [11]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests


# Function to detect the type of a variable
def detect_variable_type(df, var):
    unique_values = df[var].nunique()
    
    if unique_values == 2:
        return "binary"
    elif unique_values <= 5:  # Set a threshold for categorical variables (e.g. <= 10 unique values)
        return "categorical"
    elif pd.api.types.is_numeric_dtype(df[var]) and unique_values > 5:
        return "continuous"
    else:
        return "categorical"

# Compute p-values using logistic regression
def logistic_regression_analysis(df, var, outcome_var):
    X = df[[var]].copy()
    X = sm.add_constant(X)  # Add intercept
    y = df[outcome_var]
    
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    
    coef = result.params[var]
    p_value = result.pvalues[var]
    
    return coef, p_value

# Apply Bonferroni correction
def apply_bonferroni_correction(p_values, alpha=0.05):
    corrected_p_values = multipletests(p_values, alpha=alpha, method='bonferroni')
    return corrected_p_values[1]  # Return corrected p-values

# Summary function
def get_summary(df, group_name, variables, outcome_var):
    group_summary = []
    p_values = []  # List to store p-values
    total_patients = len(df)
    no_aki_patients = len(df[df[outcome_var] == 0])
    aki_patients = len(df[df[outcome_var] == 1])
    
    for var in variables:
        var_type = detect_variable_type(df, var)
        
        if var_type == "continuous":
            all_patients = df[var].mean(), df[var].std()
            no_aki = df[df[outcome_var] == 0][var].mean(), df[df[outcome_var] == 0][var].std()
            aki = df[df[outcome_var] == 1][var].mean(), df[df[outcome_var] == 1][var].std()

            coef, p_value = logistic_regression_analysis(df, var, outcome_var)
            p_values.append(p_value)

            group_summary.append([var, 
                                  f"{all_patients[0]:.2f} ± {all_patients[1]:.2f}", 
                                  f"{no_aki[0]:.2f} ± {no_aki[1]:.2f}", 
                                  f"{aki[0]:.2f} ± {aki[1]:.2f}",
                                  f"{coef:.4f}", 
                                  f"{p_value:.4f}"])
        
        elif var_type == "binary":
            all_patients_pos = df[var].sum()
            no_aki_pos = df[df[outcome_var] == 0][var].sum()
            aki_pos = df[df[outcome_var] == 1][var].sum()

            all_patients_perc = all_patients_pos / total_patients * 100
            no_aki_perc = no_aki_pos / no_aki_patients * 100
            aki_perc = aki_pos / aki_patients * 100

            coef, p_value = logistic_regression_analysis(df, var, outcome_var)
            p_values.append(p_value)

            group_summary.append([var, 
                                  f"{all_patients_pos} ({all_patients_perc:.1f}%)", 
                                  f"{no_aki_pos} ({no_aki_perc:.1f}%)", 
                                  f"{aki_pos} ({aki_perc:.1f}%)",
                                  f"{coef:.4f}", 
                                  f"{p_value:.4f}"])
        
        elif var_type == "categorical":
            group_summary.append([var, '', '', '', '', ''])
            
            categories = sorted(df[var].unique())  # Sort ascending
            
            for cat in categories:
                df[f"{var}_{cat}"] = (df[var] == cat).astype(int)

                all_patients_pos = df[df[var] == cat][var].count()
                no_aki_pos = df[(df[outcome_var] == 0) & (df[var] == cat)][var].count()
                aki_pos = df[(df[outcome_var] == 1) & (df[var] == cat)][var].count()

                all_patients_perc = all_patients_pos / total_patients * 100
                no_aki_perc = no_aki_pos / no_aki_patients * 100
                aki_perc = aki_pos / aki_patients * 100

                coef, p_value = logistic_regression_analysis(df, f"{var}_{cat}", outcome_var)
                p_values.append(p_value)

                group_summary.append([f"{cat}", 
                                      f"{all_patients_pos} ({all_patients_perc:.1f}%)", 
                                      f"{no_aki_pos} ({no_aki_perc:.1f}%)", 
                                      f"{aki_pos} ({aki_perc:.1f}%)",
                                      f"{coef:.4f}", 
                                      f"{p_value:.4f}"])

    return group_summary, p_values

# Print the summary including logistic regression and Bonferroni correction
def print_summary(df, groups, outcome_var='AKI_bin'):
    total_patients = len(df)
    no_aki_patients = len(df[df[outcome_var] == 0])
    aki_patients = len(df[df[outcome_var] == 1])

    print(f"### Overview ###")
    print(f"Total patients: n = {total_patients}")
    print(f"No PO-AKI: n = {no_aki_patients} ({no_aki_patients / total_patients * 100:.1f}%)")
    print(f"PO-AKI: n = {aki_patients} ({aki_patients / total_patients * 100:.1f}%)\n")

    all_p_values = []  # List to store all p-values
    group_summaries = []  # For the group summaries

    for group_name, variables in groups.items():
        print(f"### {group_name} ###")
        group_summary, p_values = get_summary(df, group_name, variables, outcome_var)
        all_p_values.extend(p_values)  # Append p-values to the overall list
        group_summaries.append(group_summary)
        print(tabulate(group_summary, headers=['Variable', f'All patients n = {total_patients}', 
                                               f'No PO-AKI n = {no_aki_patients}', 
                                               f'PO-AKI n = {aki_patients}', 
                                               'Coefficient', 'p-value'], tablefmt='pretty'))
        print("\n")

    # Bonferroni correction of all collected p-values
    corrected_p_values = apply_bonferroni_correction(all_p_values)

    # Insert and format p-values in the group summaries
    idx = 0
    for group_summary in group_summaries:
        for row in group_summary:
            if row[-1] != '':
                adjusted_p_value = corrected_p_values[idx]
                if adjusted_p_value < 0.001:
                    row.append("< 0.001")
                else:
                    row.append(f"{adjusted_p_value:.4f}")
                idx += 1

    # Print full tables with corrected p-values
    for group_name, group_summary in zip(groups.keys(), group_summaries):
        print(f"### {group_name} (mit adjustierten p-Werten) ###")
        print(tabulate(group_summary, headers=['Variable', f'All patients n = {total_patients}', 
                                               f'No PO-AKI n = {no_aki_patients}', 
                                               f'PO-AKI n = {aki_patients}', 
                                               'ß', 'p-value', 'Adjusted p-value'], tablefmt='pretty'))
        print("\n")

# Call with df_training and the predefined groups
print_summary(df_training, groups, outcome_var='AKI_bin')

### Overview ###
Total patients: n = 2286
No PO-AKI: n = 1143 (50.0%)
PO-AKI: n = 1143 (50.0%)

### Demography ###
+----------+-----------------------+--------------------+-----------------+-------------+---------+
| Variable | All patients n = 2286 | No PO-AKI n = 1143 | PO-AKI n = 1143 | Coefficient | p-value |
+----------+-----------------------+--------------------+-----------------+-------------+---------+
|   age    |     59.72 ± 16.50     |   57.71 ± 17.47    |  61.72 ± 15.22  |   0.0149    | 0.0000  |
|  height  |     171.77 ± 9.81     |   171.54 ± 9.93    |  172.01 ± 9.68  |   0.0049    | 0.2471  |
|   bmi    |     27.08 ± 6.08      |    26.78 ± 5.92    |  27.38 ± 6.24   |   0.0163    | 0.0188  |
|   sex    |     1261 (55.2%)      |    567 (49.6%)     |   694 (60.7%)   |   0.4512    | 0.0000  |
+----------+-----------------------+--------------------+-----------------+-------------+---------+


### Preoperative risk assessment ###
+-------------+-----------------------+-------

/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
print_summary(df_validation, groups, outcome_var='AKI_bin')

### Overview ###
Total patients: n = 4630
No PO-AKI: n = 4055 (87.6%)
PO-AKI: n = 575 (12.4%)

### Demography ###
+----------+-----------------------+--------------------+----------------+-------------+---------+
| Variable | All patients n = 4630 | No PO-AKI n = 4055 | PO-AKI n = 575 | Coefficient | p-value |
+----------+-----------------------+--------------------+----------------+-------------+---------+
|   age    |     58.68 ± 17.30     |   58.00 ± 17.53    | 63.48 ± 14.71  |   0.0196    | 0.0000  |
|  height  |     171.98 ± 9.75     |   172.01 ± 9.83    | 171.70 ± 9.16  |   -0.0033   | 0.4729  |
|   bmi    |     26.81 ± 5.58      |    26.83 ± 5.62    |  26.64 ± 5.34  |   -0.0064   | 0.4324  |
|   sex    |     2493 (53.8%)      |    2158 (53.2%)    |  335 (58.3%)   |   0.2046    | 0.0234  |
+----------+-----------------------+--------------------+----------------+-------------+---------+


### Preoperative risk assessment ###
+-------------+-----------------------+----------------

/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


+------------+-----------------------+--------------------+----------------+-------------+---------+
|  Variable  | All patients n = 4630 | No PO-AKI n = 4055 | PO-AKI n = 575 | Coefficient | p-value |
+------------+-----------------------+--------------------+----------------+-------------+---------+
|    nyha    |                       |                    |                |             |         |
|     0      |     4358 (94.1%)      |    3828 (94.4%)    |  530 (92.2%)   |   -0.3589   | 0.0344  |
|     1      |       21 (0.5%)       |     19 (0.5%)      |    2 (0.3%)    |   -0.2992   | 0.6879  |
|     2      |      146 (3.2%)       |     120 (3.0%)     |   26 (4.5%)    |   0.4402    | 0.0465  |
|     3      |      101 (2.2%)       |     84 (2.1%)      |   17 (3.0%)    |   0.3648    | 0.1763  |
|     4      |       4 (0.1%)        |      4 (0.1%)      |    0 (0.0%)    |  -10.5182   | 0.9671  |
|    chf     |      152 (3.3%)       |     130 (3.2%)     |   22 (3.8%)    |   0.1833    | 

In [13]:
import numpy as np
from tabulate import tabulate
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

# Function to detect the type of a variable
def detect_variable_type(df, var):
    unique_values = df[var].nunique()
    
    if unique_values == 2:
        return "binary"
    elif unique_values <= 5:  # Define a threshold for categorical variables
        return "categorical"
    elif pd.api.types.is_numeric_dtype(df[var]) and unique_values > 5:
        return "continuous"
    else:
        return "categorical"

# Compute p-values using logistic regression
def logistic_regression_analysis(df, var, outcome_var):
    X = df[[var]].copy()
    X = sm.add_constant(X)  # Add intercept
    y = df[outcome_var]
    
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    
    coef = result.params[var]
    p_value = result.pvalues[var]
    
    return coef, p_value

# Apply Bonferroni correction
def apply_bonferroni_correction(p_values, alpha=0.05):
    corrected_p_values = multipletests(p_values, alpha=alpha, method='bonferroni')
    return corrected_p_values[1]  # Return corrected p-values

# Summary function
def get_summary(df, group_name, variables, outcome_var):
    group_summary = []
    p_values = []  # List to store p-values
    total_patients = len(df)
    no_aki_patients = len(df[df[outcome_var] == 0])
    aki_patients = len(df[df[outcome_var] == 1])
    
    for var in variables:
        var_type = detect_variable_type(df, var)
        
        if var_type == "continuous":
            all_patients = df[var].mean(), df[var].std()
            no_aki = df[df[outcome_var] == 0][var].mean(), df[df[outcome_var] == 0][var].std()
            aki = df[df[outcome_var] == 1][var].mean(), df[df[outcome_var] == 1][var].std()

            coef, p_value = logistic_regression_analysis(df, var, outcome_var)
            p_values.append(p_value)

            group_summary.append([var, 
                                  f"{all_patients[0]:.2f} ± {all_patients[1]:.2f}", 
                                  f"{no_aki[0]:.2f} ± {no_aki[1]:.2f}", 
                                  f"{aki[0]:.2f} ± {aki[1]:.2f}",
                                  f"{coef:.4f}", 
                                  f"{p_value:.4f}"])
        
        elif var_type == "binary":
            all_patients_pos = df[var].sum()
            no_aki_pos = df[df[outcome_var] == 0][var].sum()
            aki_pos = df[df[outcome_var] == 1][var].sum()

            all_patients_perc = all_patients_pos / total_patients * 100
            no_aki_perc = no_aki_pos / no_aki_patients * 100
            aki_perc = aki_pos / aki_patients * 100

            coef, p_value = logistic_regression_analysis(df, var, outcome_var)
            p_values.append(p_value)

            group_summary.append([var, 
                                  f"{all_patients_pos} ({all_patients_perc:.1f}%)", 
                                  f"{no_aki_pos} ({no_aki_perc:.1f}%)", 
                                  f"{aki_pos} ({aki_perc:.1f}%)",
                                  f"{coef:.4f}", 
                                  f"{p_value:.4f}"])
        
        elif var_type == "categorical":
            group_summary.append([var, '', '', '', '', ''])
            
            categories = sorted(df[var].unique())  # Sort ascending
            
            for cat in categories:
                df[f"{var}_{cat}"] = (df[var] == cat).astype(int)

                all_patients_pos = df[df[var] == cat][var].count()
                no_aki_pos = df[(df[outcome_var] == 0) & (df[var] == cat)][var].count()
                aki_pos = df[(df[outcome_var] == 1) & (df[var] == cat)][var].count()

                all_patients_perc = all_patients_pos / total_patients * 100
                no_aki_perc = no_aki_pos / no_aki_patients * 100
                aki_perc = aki_pos / aki_patients * 100

                coef, p_value = logistic_regression_analysis(df, f"{var}_{cat}", outcome_var)
                p_values.append(p_value)

                group_summary.append([f"{cat}", 
                                      f"{all_patients_pos} ({all_patients_perc:.1f}%)", 
                                      f"{no_aki_pos} ({no_aki_perc:.1f}%)", 
                                      f"{aki_pos} ({aki_perc:.1f}%)",
                                      f"{coef:.4f}", 
                                      f"{p_value:.4f}"])

    return group_summary, p_values

# Function to export the results to an Excel file
def export_to_excel(df, groups, outcome_var='AKI_bin', file_name='summary.xlsx'):
    total_patients = len(df)
    no_aki_patients = len(df[df[outcome_var] == 0])
    aki_patients = len(df[df[outcome_var] == 1])

    all_p_values = []  # List to store all p-values
    group_summaries = []  # For the group summaries

    # Create a pandas ExcelWriter
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        for group_name, variables in groups.items():
            group_summary, p_values = get_summary(df, group_name, variables, outcome_var)
            all_p_values.extend(p_values)  # Add p-values to the overall list
            group_summaries.append(group_summary)
            
            # Shorten the sheet name if longer than 31 characters
            if len(group_name) > 31:
                group_name = group_name[:31]

            # Create a DataFrame for this group
            df_group = pd.DataFrame(group_summary, columns=['Variable', f'All patients n = {total_patients}', 
                                                            f'No PO-AKI n = {no_aki_patients}', 
                                                            f'PO-AKI n = {aki_patients}', 
                                                            'Coefficient', 'p-value'])
            # Write the group to the Excel sheet
            df_group.to_excel(writer, sheet_name=group_name, index=False)
            
        # Bonferroni correction of all collected p-values
        corrected_p_values = apply_bonferroni_correction(all_p_values)

        # Insert and format p-values in the group summaries
        idx = 0
        for group_name, group_summary in zip(groups.keys(), group_summaries):
            for row in group_summary:
                if row[-1] != '':
                    adjusted_p_value = corrected_p_values[idx]
                    if adjusted_p_value < 0.001:
                        row.append("< 0.001")
                    else:
                        row.append(f"{adjusted_p_value:.4f}")
                    idx += 1

            # Shorten the sheet name for adjusted results if longer than 31 characters
            adjusted_sheet_name = f"{group_name}_adjusted"
            if len(adjusted_sheet_name) > 31:
                adjusted_sheet_name = adjusted_sheet_name[:31]

            # Write the full table with corrected p-values to another Excel sheet
            df_group_adjusted = pd.DataFrame(group_summary, columns=['Variable', f'All patients n = {total_patients}', 
                                                                     f'No PO-AKI n = {no_aki_patients}', 
                                                                     f'PO-AKI n = {aki_patients}', 
                                                                     'ß', 'p-value', 'Adjusted p-value'])
            df_group_adjusted.to_excel(writer, sheet_name=adjusted_sheet_name, index=False)

In [15]:
export_to_excel(df_validation, groups, outcome_var='AKI_bin', file_name='summary_validationcohort.xlsx')
export_to_excel(df_training_balanced, groups, outcome_var='AKI_bin', file_name='summary_trainingbalancedcohort.xlsx')

/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/hendrikmeyer/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
